<a href="https://colab.research.google.com/github/vincentsun870530/COMP432_Anomaly_Detection/blob/final_encoder_random_matrix/comp432.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sklearn
import sklearn
import sklearn.preprocessing
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# numpy, pandas and matplotlib
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib
from matplotlib import pyplot as plt

from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import GaussianNoise

from mpl_toolkits.mplot3d import Axes3D


RANDOM_SEED = 0
np.random.seed(RANDOM_SEED)

### Dataset cleaning and reformatting

In [ ]:
data = pd.read_csv('data/PRSA_data_2010.1.1-2014.12.31.csv')
data.head(5)

In [ ]:
# Create timestamp
data["timestamp"] = pd.to_datetime(data[["year", "month", "day", "hour"]])

In [ ]:
data.drop(["No"], axis=1, inplace=True) 

In [ ]:
# Set timestamp as a new index
data = data.set_index("timestamp")
data.head(5)

In [ ]:
# Check dataset shape
print(data.shape)

In [ ]:
# Drop rows with NAN value 
data = data.dropna()
data.head(5)

In [ ]:
# Check dataset shape
print(data.shape)

In [ ]:
# Factorize string data to integer 
data.loc[:, 'cbwd'] = pd.factorize(data.loc[:, 'cbwd'])[0]
data.head(5)

### Data static summary and visualization

In [ ]:
# Summarize key columns
print(data[['pm2.5', 'DEWP', 'TEMP', 'PRES']].describe())

In [ ]:
# Show how pm2.5 value looks like
plt.figure(figsize=[16, 8])
data['pm2.5'].plot(kind='line',color='green',grid=True)
plt.title("PM2.5 with timestamp");

In [ ]:
# Show one month zoom in pm2.5 data
plt.figure(figsize=[16, 8])
data['pm2.5'][:700].plot(kind='line',color='green', grid=True);

In [ ]:
# plot the data distribution
content_list = ['pm2.5', 'TEMP', 'DEWP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir']
title_list = ['PM2.5 Concentration Distribution', 'Temperature Distribution', 'Dew Point Distribution', \
             'Pressure Distribution', 'Combined Wind Direction Distribution', 'Cumulated wind speed (m/s) Distribution', \
             'Cumulated Hours Of Snow Distribution', 'Cumulated Hours Of Rnow Distribution']

plt.figure(figsize=(18,10))
for i in range(len(content_list)):
    plt.subplot(2, 4, i+1)
    plt.hist(data[content_list[i]], bins='auto')
    plt.title(title_list[i])

In [ ]:
# PM2.5 distribution
plt.figure(figsize=[16, 8])
plt.hist(data['pm2.5'], bins='auto');

In [ ]:
# Multivariate Plots
scatter_matrix(data[['pm2.5','TEMP','DEWP','PRES']],figsize=[16,16])
plt.show()

In [ ]:
# Zoom in the relation between temp and pm2.5
plt.figure(figsize=[16, 8])
plt.scatter(data['TEMP'], data['pm2.5'], marker= "o");

In [ ]:
# Zoom in the relation between PRES and pm2.5
plt.figure(figsize=[16, 8])
plt.scatter(data['PRES'], data['pm2.5'], marker= "o");

In [ ]:
# Zoom in the relation between Iws and pm2.5
plt.figure(figsize=[16, 8])
plt.scatter(data['Iws'], data['pm2.5'], marker= "o");

### Data wrangling

In [ ]:
# Sign a new column
data = data.assign(anom=pd.Series(np.zeros(len(data), dtype=np.int)).values)
data.head(5)

In [ ]:
df25 = data.loc[:, 'pm2.5']
df25[:5]

In [ ]:
# Different rolling window for labeling
# Rolling window 12hrs
df25_12_column = df25 - df25.rolling(12).mean()
df25_12_column.name = 'diff_12hr_pm2.5'
df25_12_column.hist();

In [ ]:
# Rolling window 24hrs
df25_24_column = df25 - df25.rolling(24).mean()
df25_24_column.name = 'diff_24hr_pm2.5'
df25_24_column.hist();

In [ ]:
# Rolling window 48hrs
df25_48_column = df25 - df25.rolling(48).mean()
df25_48_column.name = 'diff_48hr_pm2.5'
df25_48_column.hist();

In [ ]:
# Rolling window 144hrs
df25_144_column = df25 - df25.rolling(144).mean()
df25_144_column.name = 'diff_144hr_pm2.5'
df25_144_column.hist();

In [ ]:
# Declear window list
window_list = [df25_12_column, df25_24_column, df25_48_column, df25_144_column]

In [ ]:
def get_new_dataframe_with_anomaly_label(original_dataset, window_column_series):
    '''
    This function gets an original pm2.5 dataframe and a pm2.5 diff series, then join it by same timestamp index. Return a dataframe contain pm2.5 dataframe with pm2.5 diff series.
    original_dataset: pandas dataframe
    window_column_series: pandas series
    '''
    newdata = pd.concat([data, window_column_series], axis=1)
    newdata.columns.values[-1] = 'diff_pm_2.5'
    newdata.loc[newdata['diff_pm_2.5'] >= 3*(newdata['diff_pm_2.5']).std(),'anom'] = 1
    plt.figure(figsize=(16,8))
    plt.plot(newdata['pm2.5'], markevery=[newdata['anom']==1], marker='x', markeredgecolor='red', color='green',markersize=10)
    print('{0} total anomaly numbers : {1}'.format(window_column_series.name,len(newdata[newdata['anom']==1])))
    plt.title('Window size : {0}'.format(window_column_series.name))
    return newdata

In [ ]:
newdata_list = []
for i in window_list:
    newdata_list.append(get_new_dataframe_with_anomaly_label(data, i))

### Data normalization

In [ ]:
# we use the first dataset in newdata_list in our project
newdata = newdata_list[3]
# drop the first few lines with NaN in diff_pm_2.5
newdata = newdata.dropna()
tag = newdata['anom'].copy()

In [ ]:
# this part is refer to "https://www.kaggle.com/robinteuwens/anomaly-detection-with-auto-encoders"
# manual parameter 
RATIO_TO_ANOMALY = 10

# splitting by class
anomaly = newdata[newdata.anom == 1]
normal = newdata[newdata.anom == 0]

# undersample clean transactions
normal_undersampled = normal.sample(
    int(len(anomaly) * RATIO_TO_ANOMALY),
    random_state=RANDOM_SEED
)

# concatenate with fraud transactions into a single dataframe
visualisation_initial = pd.concat([anomaly, normal_undersampled])
column_names = list(visualisation_initial.drop('anom', axis=1).columns)

# isolate features from labels 
features, labels = visualisation_initial.drop('anom', axis=1).values, \
                   visualisation_initial.anom.values


def tsne_scatter(features, labels, dimensions=2, save_as='graph.png'):
    if dimensions not in (2, 3):
        raise ValueError('tsne_scatter can only plot in 2d or 3d')
    # t-SNE dimensionality reduction
    features_embedded = TSNE(n_components=dimensions, random_state=RANDOM_SEED).fit_transform(features)
    # initialising the plot
    fig, ax = plt.subplots(figsize=(8,8))
    # counting dimensions
    if dimensions == 3: ax = fig.add_subplot(111, projection='3d')
    # plotting data
    ax.scatter(
        *zip(*features_embedded[np.where(labels==1)]),
        marker='o',
        color='r',
        s=2,
        alpha=0.7,
        label='Anomaly'
    )
    ax.scatter(
        *zip(*features_embedded[np.where(labels==0)]),
        marker='o',
        color='g',
        s=2,
        alpha=0.3,
        label='Normal'
    )
                         
    # storing it to be displayed later
    plt.legend(loc='best')
    plt.savefig(save_as)
    plt.show

In [ ]:
# print the t-sne
tsne_scatter(features, labels, dimensions=2, save_as='tsne_initial_2d.png')

In [ ]:
# Your code here. Aim for 2-3 lines.
from sklearn.model_selection import train_test_split
training_data, testing_data, training_data_tag, testing_data_tag = train_test_split(newdata, tag, test_size=0.2, random_state=42)
pure_training_data, validation_training_data, pure_training_tag, validation_training_tag = train_test_split(training_data, training_data_tag, test_size=0.25, random_state=42)

In [ ]:
# generate normal/anomaly data mask
normal_mask = pure_training_data['anom']==0
anomaly_mask = pure_training_data['anom']==1

normal_pure_training_tag = pure_training_tag[normal_mask]
anomaly_pure_training_tag = pure_training_tag[anomaly_mask]

# normalize the pure training data
pure_training_data=(pure_training_data-pure_training_data.mean())/pure_training_data.std()

# normalize the validation data
validation_training_data=(validation_training_data-validation_training_data.mean())/validation_training_data.std()

# normalize the testing data
testing_data = (testing_data-testing_data.mean())/testing_data.std()

# make a copy for seperate purpose
whole_data = newdata.copy()
whole_data.drop('anom',axis=1,inplace=True)
whole_data_wot = whole_data.values

In [ ]:
# the autoencoder does not need to know the tag information and some other information for tag purpose, so drop it
pure_training_data.drop('anom',axis=1,inplace=True)
pure_training_data.drop('diff_pm_2.5',axis=1,inplace=True)

validation_training_data.drop('anom',axis=1,inplace=True)
validation_training_data.drop('diff_pm_2.5',axis=1,inplace=True)

testing_data.drop('anom',axis=1,inplace=True)
testing_data.drop('diff_pm_2.5',axis=1,inplace=True)

In [ ]:
df_normal = pure_training_data[normal_mask]
df_anomaly = pure_training_data[anomaly_mask]

In [ ]:
x_normal_train = df_normal.values
X_anomaly = df_anomaly.values
x_normal_validation = validation_training_data.values
X_testing = testing_data.values

In [ ]:
%%time
model_list = []
history_list = []
hidden_layer_list = []
for outlayer_dim in range(4, 12):
    for midlayer_dim in range(3, outlayer_dim):
        for inlayer_dim in range(2, midlayer_dim):
            hidden_layer_list.append(F"{outlayer_dim}:{midlayer_dim}:{inlayer_dim}:{midlayer_dim}:{outlayer_dim}")
            model = Sequential()
            model.add(Dense(outlayer_dim, input_dim=x_normal_train.shape[1], activation='relu'))
            model.add(Dense(midlayer_dim, activation='relu')) # latent code layer
            model.add(Dense(inlayer_dim, activation='relu')) # latent code layer
            model.add(Dense(midlayer_dim, activation='relu')) 
            model.add(Dense(outlayer_dim, activation='relu'))
            model.add(Dense(x_normal_train.shape[1])) # Multiple output neurons
            model.compile(loss='mean_squared_error', optimizer='adam')
            history = model.fit(x_normal_train, x_normal_train, 
                      epochs=100, 
                      batch_size=512,
                      validation_data=(x_normal_validation, x_normal_validation),
                      shuffle=True)
            model_list.append(model)
            history_list.append(history)

In [ ]:
def printCurve(x, y, title, xlabel, ylabel):
    plt.figure()
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x, y)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
rec_score_list = []
prc_score_list = []

# for i in range(len(model_list)):
for i in range(len(model_list)):
    model = model_list[i]
    history = history_list[i]
    
    pred5 = model.predict(x_normal_validation)
    score5 = np.sqrt(metrics.mean_squared_error(pred5,x_normal_validation))
    
    # this dataframe shows the difference between the actural pm2.5 and the reconstructed pm2.5
    df_diff = x_normal_validation[:, 4] - pred5[:, 4]
    diff = np.absolute(df_diff)
    
    # actural numbers of anomaly data classified by the "rule" we set
    true_tag = validation_training_tag.values.astype(np.int)
    
    threshold = np.arange(0.1, 6, 0.1)
    rec_score = np.zeros(len(threshold))
    pred_tag = np.zeros(len(threshold))
    prc_score = np.zeros(len(threshold))
    
    for i in range(len(threshold)):
        pred_tag = diff >= threshold[i]
        rec_score[i] = sklearn.metrics.recall_score(true_tag, pred_tag)
        prc_score[i] = sklearn.metrics.precision_score(true_tag, pred_tag)
        pred_tag = np.zeros(len(threshold))
    
    rec_score_list.append(rec_score)
    prc_score_list.append(prc_score)
    
#     printCurve(rec_score, prc_score, "2 class Precision-Recall curve", "Recall", "Precision")

In [ ]:
def printCurve(x_list, y_list, title_list, xlabel, ylabel):
    plt.figure(figsize=(20,120))
    for i in range(len(x_list)):
        plt.subplot(24, 5, i+1)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.plot(x_list[i], y_list[i])
        plt.title(title_list[i] + F" {i}")

In [ ]:
rec_score_list[68], prc_score_list[68]

In [ ]:

plt.figure(figsize=(5,5))
plt.scatter(rec_score_list[68], prc_score_list[68], s=1)

In [ ]:
from sklearn.metrics import auc
temp = auc(rec_score_list[68], prc_score_list[68])
print(temp)

In [ ]:
len(prc_score_list)

In [ ]:
printCurve(rec_score_list, prc_score_list, hidden_layer_list, "Recall", "Precision")

In [ ]:
# auroc_list = auc(prc_score_list)

auc_list = []
for rec, prc in zip(rec_score_list, prc_score_list):
  auc_list.append(auc(rec, prc))

max_auroc = max(auroc_list)
maxauroc_index_list = [i for i, j in enumerate(auroc_list) if j == max_auroc]
best_classifier_index = maxauroc_index_list[0]

print(F"the best classifier is NO.{best_classifier_index} and the max auroc is {max_auroc}")

In [ ]:
prc_score_list[87].sum(), prc_score_list[0]

In [ ]:
rec_score_list[0]

In [ ]:
 best_classifier_index

In [ ]:
model_list[36].summary()

In [ ]:
# best_model_index = best_classifier_index
best_model_index = 36
print(F"the best model is {hidden_layer_list[best_model_index]}")
model_list[best_model_index].summary()
print(F"model {best_model_index} training loss = {history_list[best_model_index].history['loss'][-1]}")
print(F"model {best_model_index} validation loss = {history_list[best_model_index].history['val_loss'][-1]}")

In [ ]:
model = model_list[best_model_index]
history = history_list[best_model_index]

In [ ]:
# print the loss with epochs
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
pred1 = model.predict(X_testing)
score1 = np.sqrt(metrics.mean_squared_error(pred1,X_testing))
print(f"Test smaples Score (RMSE): {score1}")

In [ ]:
training_data2=(training_data-training_data.mean())/training_data.std()
training_data2.drop('anom',axis=1,inplace=True)
training_data2.drop('diff_pm_2.5',axis=1,inplace=True)
pred2 = model.predict(training_data2)
score2 = np.sqrt(metrics.mean_squared_error(pred2,training_data2))
print(f"Test smaples Score (RMSE): {score2}")

In [ ]:
pred3 = model.predict(X_anomaly)
score3 = np.sqrt(metrics.mean_squared_error(pred3,X_anomaly))
print(f"Test smaples Score (RMSE): {score3}")

In [ ]:
training_data.shape, X_testing.shape


In [ ]:
# this dataframe shows the difference between the actural pm2.5 and the reconstructed pm2.5
df_diff = X_testing[:, 4] - pred1[:, 4]
diff = np.absolute(df_diff)
# diff = df_diff

In [ ]:
# actural numbers of anomaly data classified by the "rule" we set
true_tag = testing_data_tag

In [ ]:
threshold = np.arange(0.1, 6, 0.1)
rec_score = np.zeros(len(threshold))
# pred_tag = np.zeros(len(threshold))
prc_score = np.zeros(len(threshold))

In [ ]:

#         plt.xlabel(xlabel)
#         plt.ylabel(ylabel)
#         plt.plot(x_list[i], y_list[i])
#         plt.title(title_list[i])

In [ ]:
pred_tag_list = []
for i in range(len(threshold)):
    pred_tag = (diff >= threshold[i])
    pred_tag_list.append(pred_tag)
    rec_score[i] = sklearn.metrics.recall_score(true_tag, pred_tag)
    prc_score[i] = sklearn.metrics.precision_score(true_tag, pred_tag)
    pred_tag = np.zeros(len(threshold))

In [ ]:
for i in range(len(pred_tag_list)):
    # plt.title(F"Threshold value is {threshold[i]}")
    cm = confusion_matrix(true_tag, pred_tag_list[i])
    print(F"Threshold value is {threshold[i]} (index {i})")
    print(cm);

In [ ]:
# from the above printed information we choose the NO.29 threshold value in list(3.0)

In [ ]:
def printCuv(x, y, title, xlabel, ylabel):
    plt.figure()
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x, y)

In [ ]:
printCuv(rec_score, prc_score, "2 class Precision-Recall curve", "Recall", "Precision")

In [ ]:
printCuv(threshold, prc_score, "2 class Threshold-Precision curve", "threshold", "Precision")

In [ ]:
printCuv(threshold, rec_score, "2 class Threshold-Recall curve", "threshold", "Recall")

In [ ]:
print(F"threshold value we choose is {threshold[29]}, with precision {prc_score_list[36][29]} and recall {rec_score_list[36][29]}")